In [1]:
import yfinance as yf
import alpaca_trade_api
import selenium
import time
from pytz import timezone
from datetime import datetime
from msedge.selenium_tools import EdgeOptions, Edge
import smtplib

In [2]:
#-----------authentication---------------
api_key = 'PKQBLGJ7YSSH8RBKCDU0'
secret_key = 'Kwx4sHT38Gv8nlJjxWfFv1EtxpTgNkc6ivqpFIZq'
endpoint = 'https://paper-api.alpaca.markets'
api =  alpaca_trade_api.REST(api_key, secret_key, endpoint, api_version='v2')
email_add = 'byethost.panel@gmail.com'
email_pass = 'bybjugzqzfzjrimz'

In [3]:
def get_stock_list():
    tickers=[]
    url = 'https://finance.yahoo.com/gainers'

    options = EdgeOptions()
    options.use_chromium = True
    options.add_argument("headless")
    options.add_argument("disable-gpu")
    driver = Edge(options = options)
    driver.get(url)
    driver.implicitly_wait(5)

    for x in range(1,6):
        per_change = driver.find_element_by_xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{x}]/td[5]/fin-streamer/span').text
        l = len(per_change)
        value = float(per_change[1:l-1])
        #value = ''.join(element)
        if float(value) > 0:
            tickers.append(driver.find_element_by_xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{x}]/td[1]/a').text)

    element = driver.find_element_by_xpath('//*[@id="scr-res-table"]/div[1]/table/thead/tr/th[4]')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.click()
    time.sleep(2)

    for x in range(1,6):
        element = driver.find_element_by_xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{x}]/td[1]/a').text
        change = driver.find_element_by_xpath(f'//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{x}]/td[4]/fin-streamer/span').text
        l = len(change)
        value = float(change[1:])
        if element not in tickers and value > 0:
            tickers.append(element)

    return tickers

In [4]:
def get_current_price(symbol):
    ticker_data = yf.Ticker(symbol)
    data = ticker_data.history(period='1d')
    return data['Close'][0]

In [5]:
def place_order(sym, current_price):
    if api.get_asset(sym).tradable:
        api.submit_order(
        symbol=sym,
        qty=1,
        side='buy',
        type='market',      #market g/b??
        time_in_force='gtc',
        order_class='bracket',
        stop_loss={'stop_price': current_price * 0.90,
                'limit_price':  current_price * 0.90},
        take_profit={'limit_price': current_price * 1.05}
        )

In [6]:
def send_email(body):
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(email_add, email_pass)
        msg = f'Subject: Trading Bot Alert\n\n{body}'
        smtp.sendmail(email_add, email_add, msg)

In [7]:
def main(stop_time):
    while True:
        l_est = timezone('America/New_York')
        l_time_obj = datetime.now(l_est).strftime("%H:%M:%S")
        l_time = datetime.strptime(l_time_obj, "%H:%M:%S")
        if l_time >= stop_time:
            break

        stock_list = get_stock_list()

        portfolio = api.list_positions()
        owned = []
        for x in portfolio:
            owned.append(x.symbol)

        for i in stock_list:
            if i not in owned:
                try:
                    current_price = get_current_price(i)
                    place_order(i,current_price)
                except:
                    continue

        time.sleep(3600)

In [8]:
clock = api.get_clock()
est = timezone('America/New_York')
open_time = datetime.strptime('09:30:00', "%H:%M:%S")
exec_time = datetime.strptime('09:59:00', "%H:%M:%S")
stop_time = datetime.strptime('15:32:00', "%H:%M:%S")

while True:
    weekday = datetime.today().astimezone(est).weekday()
    curr_time_obj = datetime.now(est).strftime("%H:%M:%S")
    curr_time = datetime.strptime(curr_time_obj, "%H:%M:%S")

    if weekday <= 4:
        if curr_time > open_time and clock.is_open == False:
            send_email('Market Closed Today.')
            time.sleep(84540)

        if curr_time > open_time and clock.is_open == True and curr_time >= exec_time:
            main(stop_time)

            if curr_time >= stop_time:
                account = api.get_account()
                balance_change = float(account.equity) - float(account.last_equity)

                held_stocks = api.list_orders(status='open', limit=500, nested=True)
                owned = ''
                owned = None
                for x in held_stocks:
                    owned += x.symbol + ', '

                send_email(f'Today\'s portfolio balance change: ${balance_change}\n\nCurrently owned stocks: {owned}')
                time.sleep(64200)
    else:
        time.sleep(86400)